In [69]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [70]:
from google.colab import files
uploaded = files.upload()

Saving Text_Classifier.csv to Text_Classifier (3).csv


In [71]:
data = pd.read_csv("Text_Classifier1.csv")
print(data)

   Doc              Words Author
0    1     W1 W2 W3 W4 W5      C
1    2        W1 W1 W4 W3     C 
2    3           W1 W2 W5      C
3    4     W5 W6 W1 W2 W3      W
4    5           W4 W5 W6      W
5    6           W4 W6 W3      F
6    7  W2 W2 W4 W3 W5 W5      F


In [72]:
data.head()

,Doc,Words,Author
0,1,W1 W2 W3 W4 W5,C
1,2,W1 W1 W4 W3,C
2,3,W1 W2 W5,C
3,4,W5 W6 W1 W2 W3,W
4,5,W4 W5 W6,W


In [73]:
data.shape

(7, 3)

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Doc     7 non-null      int64 
 1   Words   7 non-null      object
 2   Author  7 non-null      object
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes


In [75]:
data.Author.value_counts()

C     2
W     2
F     2
C     1
Name: Author, dtype: int64

In [76]:
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English


# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [77]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [78]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
print(bow_vector)

CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f600c20a940>)


In [79]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [80]:
from sklearn.model_selection import train_test_split

X = data['Words'] # the features we want to analyze
ylabels = data['Author'] # the labels, or answers, we want to test against
print(X)
print(ylabels)

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

0       W1 W2 W3 W4 W5
1          W1 W1 W4 W3
2             W1 W2 W5
3       W5 W6 W1 W2 W3
4             W4 W5 W6
5             W4 W6 W3
6    W2 W2 W4 W3 W5 W5
Name: Words, dtype: object
0     C
1    C 
2     C
3     W
4     W
5     F
6     F
Name: Author, dtype: object


In [81]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import HashingVectorizer

hash_vector = HashingVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', hash_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f600aa5f490>),
                ('vectorizer',
                 HashingVectorizer(tokenizer=<function spacy_tokenizer at 0x7f600c20a940>)),
                ('classifier', LogisticRegression())])

In [82]:
New_Value = ["w1 w4 w6 w5 w3"]
predicted1 = pipe.predict(New_Value) #New data
print(predicted1)

['W']


In [83]:
#from sklearn import metrics
# Predicting with a test dataset
#predicted = pipe.predict(X_test)

# Model Accuracy
#print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))